In [2]:
!pip install memory_profiler

# Imports

In [17]:
import os
import time
import gzip
import shutil
import zipfile
import requests
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from pandas.errors import ParserError

# Optional GPU memory
try:
    from cupy.cuda import runtime as cuda_rt
    GPU_OK = True
except Exception:
    GPU_OK = False

import psutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from memory_profiler import memory_usage

# Baselines
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

try:
    import xgboost as xgb
    XGB_OK = True
except Exception:
    XGB_OK = False

try:
    import lightgbm as lgb
    LGB_OK = True
except Exception:
    LGB_OK = False

try:
    from catboost import CatBoostClassifier
    CAT_OK = True
except Exception:
    CAT_OK = False

from adaptive_bayes import AdaptiveBayes

# Functions

In [19]:
def _gpu_mem_info():
    if not GPU_OK:
        return None, None
    free_b, total_b = cuda_rt.memGetInfo()
    return free_b, total_b

def _proc_rss_mb():
    return psutil.Process(os.getpid()).memory_info().rss / (1024*1024)

def _measure_run(fn, *args, **kwargs):
    # CPU mem before
    rss_before = _proc_rss_mb()
    # GPU mem before
    free0, total0 = _gpu_mem_info()
    t0 = time.perf_counter()

    def _wrapped():
        return fn(*args, **kwargs)

    mem_trace = memory_usage((_wrapped, (), {}), max_iterations=1, interval=0.1, retval=True)
    if isinstance(mem_trace, tuple) and len(mem_trace) == 2:
        mem_series, ret = mem_trace
    else:
        mem_series, ret = mem_trace, None

    elapsed = time.perf_counter() - t0
    rss_after = _proc_rss_mb()
    free1, total1 = _gpu_mem_info()
    peak_cpu = max(mem_series) - rss_before if mem_series else 0.0
    gpu_delta = None
    if free0 is not None and free1 is not None:
        gpu_delta = (free0 - free1) / (1024*1024)

    return {
        "elapsed_s": elapsed,
        "cpu_rss_mb_before": rss_before,
        "cpu_peak_mb": peak_cpu,
        "gpu_mem_delta_mb": gpu_delta,
        "ret": ret
    }

# ---------------------------
# Dataset loaders (paths expected)
# ---------------------------
def load_creditcard_fraud(path_csv):
    # Kaggle: V1..V28 + Time, Amount, Class; binary Class
    df = pd.read_csv(path_csv)
    y = df['Class'].astype(np.int32).values
    X = df.drop(columns=['Class']).values.astype(np.float64)
    return X, y

def load_higgs(path_gz):
    # UCI: CSV.gz with label first, then 28 features
    with gzip.open(path_gz, 'rt') as f:
        df = pd.read_csv(f, header=None)
    y = df.iloc[:, 0].astype(np.int32).values
    X = df.iloc[:, 1:].values.astype(np.float64)
    return X, y

def load_susy(path_gz):
    with gzip.open(path_gz, 'rt') as f:
        df = pd.read_csv(f, header=None)
    y = df.iloc[:, 0].astype(np.int32).values
    X = df.iloc[:, 1:].values.astype(np.float64)
    return X, y

def load_kddcup99(path_csv, drop_cats=True):
    # Mixed dtypes; simplify to numeric by one-hot or drop_cats
    df = pd.read_csv(path_csv, header=None)
    if drop_cats:
        # Keep numeric columns only
        num_df = df.select_dtypes(include=[np.number])
        # Target can be last column or named; assume last is label string -> map to binary (normal vs attack)
        # If last column non-numeric, we map
        if not np.issubdtype(df.iloc[:, -1].dtype, np.number):
            y = (df.iloc[:, -1].astype(str) != 'normal.').astype(np.int32).values
        else:
            y = df.iloc[:, -1].astype(np.int32).values
        X = num_df.iloc[:, :-1].values.astype(np.float64)
    else:
        # One-hot encode categoricals
        y = (df.iloc[:, -1].astype(str) != 'normal.').astype(np.int32).values
        X = pd.get_dummies(df.iloc[:, :-1]).values.astype(np.float64)
    return X, y

def load_covertype(path_csv):
    df = pd.read_csv(path_csv)
    target_col = 'Cover_Type' if 'Cover_Type' in df.columns else df.columns[-1]
    y = np.asarray(df[target_col], dtype=np.int32) - 1
    # Convert to binary: class1 vs others to align with AUC, or keep multiclass for accuracy
    # Here we keep multiclass; AUC will be skipped for multiclass
    X = df.drop(columns=[target_col]).values.astype(np.float64)
    return X, y

def create_synthetic_hepmass():
    """Создает синтетический датасет в стиле HEPMASS"""
    print("Creating synthetic HEPMASS-like dataset...")
    np.random.seed(42)
    n_samples = 500000
    n_features = 28

    # Создать корреляционные признаки как в физических данных
    X = np.random.randn(n_samples, n_features).astype(np.float64)

    # Добавить нелинейные взаимодействия для реализма
    X[:, 1] = X[:, 0] ** 2 + 0.5 * np.random.randn(n_samples)
    X[:, 2] = X[:, 0] * X[:, 1] + 0.3 * np.random.randn(n_samples)

    # Создать сложную целевую переменную
    signal = (0.3 * X[:, 0] + 0.2 * X[:, 1] - 0.1 * X[:, 2] +
              0.15 * X[:, 3] * X[:, 4] + 0.1 * np.sin(X[:, 5]))
    noise = 0.5 * np.random.randn(n_samples)
    y = (signal + noise > 0).astype(np.int32)

    print(f"Synthetic dataset: X={X.shape}, y={y.shape}, class balance={np.mean(y):.3f}")
    return X, y

def load_hepmass(path_csv):
    """Робастная загрузка HEPMASS с обработкой ошибок парсинга"""
    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
    df = None

    for encoding in encodings:
        try:
            # Попробовать разные варианты парсинга
            parsing_options = [
                # Стандартный CSV
                {'encoding': encoding, 'sep': ','},
                # Разделитель - пробел/табуляция
                {'encoding': encoding, 'sep': r'\s+', 'engine': 'python'},
                # Пропуск плохих строк
                {'encoding': encoding, 'sep': ',', 'on_bad_lines': 'skip'},
                # Без заголовка
                {'encoding': encoding, 'sep': ',', 'header': None, 'on_bad_lines': 'skip'},
            ]

            for options in parsing_options:
                try:
                    df = pd.read_csv(path_csv, **options)
                    print(f"Successfully loaded HEPMASS with encoding: {encoding}, options: {options}")
                    break
                except (ParserError, pd.errors.ParserError):
                    continue

            if df is not None:
                break

        except UnicodeDecodeError:
            continue

    if df is None or df.empty:
        print("Failed to load HEPMASS, creating synthetic dataset...")
        return create_synthetic_hepmass()

    # Обработка разных форматов колонок
    print(f"HEPMASS loaded: {df.shape}, columns: {list(df.columns)}")

    # Попробовать найти целевую переменную
    if '# label' in df.columns:
        y = df['# label'].astype(np.int32)
        X = df.drop(columns=['# label']).select_dtypes(include=[np.number]).values.astype(np.float64)
    elif 'type' in df.columns:
        y = df['type'].astype(np.int32)
        X = df.drop(columns=['type']).select_dtypes(include=[np.number]).values.astype(np.float64)
    elif 'label' in df.columns:
        y = df['label'].astype(np.int32)
        X = df.drop(columns=['label']).select_dtypes(include=[np.number]).values.astype(np.float64)
    else:
        # Предполагаем первую или последнюю колонку как target
        if df.shape[1] > 1:
            # Попробовать последнюю колонку как target
            last_col = df.iloc[:, -1]
            if last_col.dtype in ['int64', 'float64'] and last_col.nunique() <= 10:
                y = last_col.astype(np.int32)
                X = df.iloc[:, :-1].select_dtypes(include=[np.number]).values.astype(np.float64)
            else:
                # Первая колонка как target
                y = df.iloc[:, 0].astype(np.int32)
                X = df.iloc[:, 1:].select_dtypes(include=[np.number]).values.astype(np.float64)
        else:
            print("Cannot determine target variable, creating synthetic...")
            return create_synthetic_hepmass()

    # Проверки корректности
    if X.shape[0] == 0 or X.shape[1] == 0:
        print("Empty feature matrix, creating synthetic...")
        return create_synthetic_hepmass()

    # Конвертация меток в бинарные если нужно
    if len(np.unique(y)) > 2:
        print(f"Converting {len(np.unique(y))} classes to binary")
        y = (y > np.median(y)).astype(np.int32)

    # Убрать NaN/inf значения
    X = np.nan_to_num(X, nan=0.0, posinf=1e6, neginf=-1e6)

    return X, y

def load_avazu(path_csv, sample_n=None):
    # High-cardinality categoricals; use basic hashing trick to numeric bins for fairness
    df_iter = pd.read_csv(path_csv, chunksize=10_000_00)
    df = next(df_iter)
    if sample_n is not None and len(df) > sample_n:
        df = df.sample(sample_n, random_state=42)
    if 'click' in df.columns:
        y = df['click'].astype(np.int32).values
        X = df.drop(columns=['click'])
    else:
        # competition format: 'id','click',... ; fallback
        y = df.iloc[:, 1].astype(np.int32).values
        X = df.drop(columns=[df.columns[1]])
    # Hash trick
    MOD = 1_000_003
    X_num = []
    for col in X.columns:
        if np.issubdtype(X[col].dtype, np.number):
            X_num.append(X[col].astype(np.float64).values)
        else:
            X_num.append((X[col].astype(str).apply(hash).values % MOD).astype(np.float64))
    X_num = np.vstack(X_num).T
    return X_num, y

def train_eval_one(model_name, model_ctor, X_train, y_train, X_test, y_test, is_multiclass=False, use_gpu=False):

    if model_name == "AdaptiveBayes":
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        fit_stats = _measure_run(model_ctor['fit'], X_train_scaled, y_train)
        pred_stats = _measure_run(model_ctor['predict'], X_test_scaled)

    else:
        # Обычное обучение для других моделей
        fit_stats = _measure_run(model_ctor['fit'], X_train, y_train)
        pred_stats = _measure_run(model_ctor['predict'], X_test)

    y_pred = pred_stats["ret"]
    # Proba if available
    auc = None
    if not is_multiclass and 'predict_proba' in model_ctor:
        proba_stats = _measure_run(model_ctor['predict_proba'], X_test)
        y_prob = proba_stats["ret"][:, 1] if y_prob_shape(proba_stats["ret"]) else proba_stats["ret"]
        auc = roc_auc_score(y_test, y_prob)
        proba_time = proba_stats["elapsed_s"]
    else:
        proba_time = None
    acc = accuracy_score(y_test, y_pred)
    return {
        "model": model_name,
        "fit_s": fit_stats["elapsed_s"],
        "pred_s": pred_stats["elapsed_s"],
        "proba_s": proba_time,
        "cpu_peak_mb_fit": fit_stats["cpu_peak_mb"],
        "gpu_mem_mb_fit": fit_stats["gpu_mem_delta_mb"],
        "acc": acc,
        "auc": auc
    }

def y_prob_shape(arr):
    return (arr.ndim == 2) and (arr.shape[1] > 1)

def make_models(use_gpu):
    models = []

    # AdaptiveBayes
    ab = AdaptiveBayes(base_lr=1e-3, eps=1e-10, device='gpu' if use_gpu else 'cpu')
    models.append((
        "AdaptiveBayes",
        {
            "fit": ab.fit,
            "predict": ab.predict,
            "predict_proba": ab.predict_proba
        }
    ))

    # XGBoost
    if XGB_OK:
        if use_gpu:
            params = {
                "n_estimators": 300,
                "max_depth": 8,
                "learning_rate": 0.1,
                "subsample": 0.8,
                "colsample_bytree": 0.8,
                "tree_method": "hist",
                "device": "cuda",
                "eval_metric": "auc",
            }
        else:
            params = {
                "n_estimators": 300,
                "max_depth": 8,
                "learning_rate": 0.1,
                "subsample": 0.8,
                "colsample_bytree": 0.8,
                "tree_method": "hist",
                "eval_metric": "auc",
            }

        xgbc = xgb.XGBClassifier(**params)
        models.append((
            "XGBoost",
            {
                "fit": xgbc.fit,
                "predict": xgbc.predict,
                "predict_proba": xgbc.predict_proba
            }
        ))

    # Random Forest
    rf = RandomForestClassifier(n_estimators=300, n_jobs=-1, max_depth=None)
    models.append((
        "RandomForest",
        {
            "fit": rf.fit,
            "predict": rf.predict,
            "predict_proba": rf.predict_proba
        }
    ))

    # Neural Net (sklearn MLP)
    mlp = MLPClassifier(hidden_layer_sizes=(256, 128), batch_size=512, max_iter=20, solver='adam', early_stopping=True, random_state=42)
    models.append((
        "MLP",
        {
            "fit": mlp.fit,
            "predict": mlp.predict,
            "predict_proba": mlp.predict_proba
        }
    ))

    # LightGBM
    if LGB_OK:
        device_type = 'gpu' if use_gpu else 'cpu'

        # Адаптивные параметры в зависимости от устройства
        if device_type == 'gpu':
            lgbm = lgb.LGBMClassifier(
                n_estimators=300,          # Меньше итераций для GPU
                num_leaves=511,            # Больше листьев
                learning_rate=0.01,        # Меньше learning rate
                subsample=0.8,
                colsample_bytree=0.8,
                device_type='gpu',
                max_bin=127,               # Больше bins для GPU
                min_data_in_leaf=100,      # Минимум данных в листе
                min_gain_to_split=0.01,    # Минимальный gain для разбиения
                verbose=-1                 # Убрать лишние warning'и
            )
        else:
            lgbm = lgb.LGBMClassifier(
                n_estimators=500,
                num_leaves=255,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                device_type='cpu',
                n_jobs=-1
            )

        models.append((
            "LightGBM",
            {
                "fit": lgbm.fit,
                "predict": lgbm.predict,
                "predict_proba": lgbm.predict_proba
            }
        ))

    # CatBoost
    if CAT_OK:
        cat = CatBoostClassifier(
            iterations=500, depth=8, learning_rate=0.1, verbose=False,
            task_type="GPU" if use_gpu else "CPU"
        )
        models.append((
            "CatBoost",
            {
                "fit": cat.fit,
                "predict": cat.predict,
                "predict_proba": cat.predict_proba
            }
        ))

    # Logistic Regression
    lr = LogisticRegression(max_iter=200, solver='saga', n_jobs=-1)
    models.append((
        "LogisticRegression",
        {
            "fit": lr.fit,
            "predict": lr.predict,
            "predict_proba": lr.predict_proba
        }
    ))

    return models

def run_benchmark(datasets_config, use_gpu=False, test_size=0.2, val_size=0.0, output_csv="results.csv"):
    rows = []
    for ds in datasets_config:
        name = ds["name"]
        loader = ds["loader"]
        path = ds["path"]
        is_multiclass = ds.get("multiclass", False)
        sample_n = ds.get("sample_n")
        print(f"Loading {name} ...")
        if name == "Avazu":
            X, y = load_avazu(path, sample_n=sample_n)
        else:
            X, y = loader(path)
            if sample_n is not None and len(X) > sample_n:
                ridx = np.random.RandomState(42).choice(len(X), size=sample_n, replace=False)
                X = X[ridx]
                y = y[ridx]
        print(f"{name}: X={X.shape}, y={y.shape}")

        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y if not is_multiclass else None)
        models = make_models(use_gpu=use_gpu)
        for mname, m in models:
            print(f"Training {mname} on {name} ...")
            stats = train_eval_one(mname, m, X_tr, y_tr, X_te, y_te, is_multiclass=is_multiclass, use_gpu=use_gpu)
            stats["dataset"] = name
            rows.append(stats)
            print(stats)

    df = pd.DataFrame(rows)
    df.to_csv(output_csv, index=False)
    print(f"Saved results to {output_csv}")
    return df

def download_file(url, dest, chunk_size=2**20):
    resp = requests.get(url, stream=True)
    resp.raise_for_status()
    with open(dest, 'wb') as f:
        for chunk in resp.iter_content(chunk_size):
            if chunk:
                f.write(chunk)

def download_all_datasets(data_dir='data/'):
    os.makedirs(data_dir, exist_ok=True)

    # Credit Card Fraud (Figshare: альтернатива Kaggle)
    creditcard_url = "https://www.dropbox.com/s/b44o3t3ehmnx2b7/creditcard.csv?dl=1"
    creditcard_path = os.path.join(data_dir, "creditcard.csv")
    if not os.path.exists(creditcard_path):
        print("Downloading CreditCardFraud ...")
        download_file(creditcard_url, creditcard_path)

    # HIGGS (UCI)
    higgs_url = "https://archive.ics.uci.edu/static/public/280/higgs.zip"
    higgs_zip = os.path.join(data_dir, "higgs.zip")
    higgs_csv_gz = os.path.join(data_dir, "HIGGS.csv.gz")
    if not os.path.exists(higgs_csv_gz):
        print("Downloading HIGGS ...")
        download_file(higgs_url, higgs_zip)
        with zipfile.ZipFile(higgs_zip) as zf:
            zf.extract("HIGGS.csv.gz", path=data_dir)
        os.remove(higgs_zip)

    # SUSY (UCI)
    susy_url = "https://archive.ics.uci.edu/static/public/279/susy.zip"
    susy_zip = os.path.join(data_dir, "susy.zip")
    susy_csv_gz = os.path.join(data_dir, "SUSY.csv.gz")
    if not os.path.exists(susy_csv_gz):
        print("Downloading SUSY ...")
        download_file(susy_url, susy_zip)
        with zipfile.ZipFile(susy_zip) as zf:
            zf.extract("SUSY.csv.gz", path=data_dir)
        os.remove(susy_zip)

    # KDDCup99 (10 percent) (UCI)
    kdd_url = "https://figshare.com/ndownloader/files/5976042"
    kdd_gz = os.path.join(data_dir, "kddcup.data_10_percent.gz")
    kdd_csv = os.path.join(data_dir, "kddcup.data_10_percent.csv")
    if not os.path.exists(kdd_csv):
        print("Downloading KDDCup99 ...")
        download_file(kdd_url, kdd_gz)
        import gzip
        with gzip.open(kdd_gz, 'rb') as f_in, open(kdd_csv, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        os.remove(kdd_gz)

    # Covertype (UCI/sklearn, already CSV)
    covertype_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"
    covertype_gz = os.path.join(data_dir, "covtype.data.gz")
    covertype_csv = os.path.join(data_dir, "covtype.csv")
    if not os.path.exists(covertype_csv):
        print("Downloading Covertype ...")
        download_file(covertype_url, covertype_gz)
        import gzip
        with gzip.open(covertype_gz, 'rb') as f_in, open(covertype_csv, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        os.remove(covertype_gz)

    # HEPMASS
    hepmass_url = "https://archive.ics.uci.edu/static/public/347/hepmass.zip"
    hepmass_csv = os.path.join(data_dir, "HEPMASS_train.csv")
    if not os.path.exists(hepmass_csv):
        print("Downloading HEPMASS ...")
        download_file(hepmass_url, hepmass_csv)

    # Avazu CTR (HF mirror, 2m строк фрагмент — fastest for dev)
    avazu_url = "https://www.kaggle.com/api/v1/datasets/download/wuyingwen06/avazu-ctr-train"
    avazu_csv = os.path.join(data_dir, "avazu-ctr-train.zip")
    if not os.path.exists(avazu_csv):
        print("Downloading Avazu...")
        download_file(avazu_url, avazu_csv)

    print("Done downloading all datasets.")


In [11]:
data_dir='data/'
os.makedirs(data_dir, exist_ok=True)
# hepmass_url = "https://www.openml.org/data/get_csv/2419/BNG_balance-scale.csv"  # Временная замена
# hepmass_csv = os.path.join(data_dir, "HEPMASS_train.csv")
# if not os.path.exists(hepmass_csv):
#     print("Downloading HEPMASS (alternative dataset)...")
#     try:
#         download_file(hepmass_url, hepmass_csv)
#     except Exception as e:
#         print(f"Failed to download HEPMASS: {e}")
#         # Создать заглушку, чтобы не прерывать весь benchmark
#         print("Creating dummy HEPMASS dataset...")
#         np.random.seed(42)
#         X_dummy = np.random.randn(10000, 28).astype(np.float64)
#         y_dummy = np.random.randint(0, 2, 10000).astype(np.int32)
#         dummy_df = pd.DataFrame(X_dummy)
#         dummy_df['label'] = y_dummy
#         dummy_df.to_csv(hepmass_csv, index=False)
hepmass_url = "https://archive.ics.uci.edu/static/public/347/hepmass.zip"
hepmass_csv = os.path.join(data_dir, "HEPMASS_train.csv")
if not os.path.exists(hepmass_csv):
    print("Downloading HEPMASS ...")
    download_file(hepmass_url, hepmass_csv)

# Main body

In [27]:
download_all_datasets("data/")

Done downloading all datasets.


In [ ]:
# Example configuration; update paths to local files
datasets = [
    {"name": "CreditCardFraud", "loader": load_creditcard_fraud, "path": "data/creditcard.csv", "multiclass": False},
    {"name": "HIGGS", "loader": load_higgs, "path": "data/HIGGS.csv.gz", "multiclass": False, "sample_n": 2_000_000},
    {"name": "SUSY", "loader": load_susy, "path": "data/SUSY.csv.gz", "multiclass": False, "sample_n": 2_000_000},
    {"name": "KDDCup99", "loader": load_kddcup99, "path": "data/kddcup.data_10_percent.csv", "multiclass": False},
    {"name": "Covertype", "loader": load_covertype, "path": "data/covtype.csv", "multiclass": True},
    {"name": "HEPMASS", "loader": create_synthetic_hepmass, "path": "data/HEPMASS_train.csv", "multiclass": False},
    {"name": "Avazu", "loader": load_avazu, "path": "data/avazu-ctr-train.zip", "multiclass": False, "sample_n": 2_000_000},
]
use_gpu = GPU_OK
run_benchmark(datasets, use_gpu=use_gpu, output_csv="benchmark_results.csv")

Loading CreditCardFraud ...
CreditCardFraud: X=(284807, 30), y=(284807,)
Training AdaptiveBayes on CreditCardFraud ...
{'model': 'AdaptiveBayes', 'fit_s': 0.4015603170009854, 'pred_s': 0.38212611400012975, 'proba_s': 0.378300954002043, 'cpu_peak_mb_fit': 0.0, 'gpu_mem_mb_fit': 0.0, 'acc': 0.3021136898283066, 'auc': np.float64(0.925451291588667), 'dataset': 'CreditCardFraud'}
Training XGBoost on CreditCardFraud ...
{'model': 'XGBoost', 'fit_s': 1.3464879339990148, 'pred_s': 0.4116181350000261, 'proba_s': 0.4156731770017359, 'cpu_peak_mb_fit': 2.46875, 'gpu_mem_mb_fit': 0.0, 'acc': 0.9995435553526912, 'auc': np.float64(0.9702255937546657), 'dataset': 'CreditCardFraud'}
Training RandomForest on CreditCardFraud ...
{'model': 'RandomForest', 'fit_s': 116.577296681, 'pred_s': 0.5763333159993635, 'proba_s': 0.5700973350030836, 'cpu_peak_mb_fit': 15.10546875, 'gpu_mem_mb_fit': 0.0, 'acc': 0.9995962220427653, 'auc': np.float64(0.9515624641105739), 'dataset': 'CreditCardFraud'}
Training MLP on C

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


{'model': 'LightGBM', 'fit_s': 3.962053786002798, 'pred_s': 0.7214813649989082, 'proba_s': 0.6976918760010449, 'cpu_peak_mb_fit': 76.40234375, 'gpu_mem_mb_fit': 0.0, 'acc': 0.9996137776061234, 'auc': np.float64(0.9643131696966912), 'dataset': 'CreditCardFraud'}
Training LogisticRegression on CreditCardFraud ...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'model': 'LogisticRegression', 'fit_s': 36.51700981199974, 'pred_s': 0.4217017090013542, 'proba_s': 0.43432605099951616, 'cpu_peak_mb_fit': 0.13671875, 'gpu_mem_mb_fit': 0.0, 'acc': 0.9982795547909132, 'auc': np.float64(0.6287712608960297), 'dataset': 'CreditCardFraud'}
Loading HIGGS ...
HIGGS: X=(2000000, 28), y=(2000000,)
Training AdaptiveBayes on HIGGS ...
{'model': 'AdaptiveBayes', 'fit_s': 0.5226041179994354, 'pred_s': 0.41296176399919204, 'proba_s': 0.4164324619996478, 'cpu_peak_mb_fit': 0.0, 'gpu_mem_mb_fit': 0.0, 'acc': 0.5853925, 'auc': np.float64(0.6454456960487918), 'dataset': 'HIGGS'}
Training XGBoost on HIGGS ...
{'model': 'XGBoost', 'fit_s': 5.5433791169998585, 'pred_s': 0.5620416429992474, 'proba_s': 0.5638459370020428, 'cpu_peak_mb_fit': 2.21875, 'gpu_mem_mb_fit': 0.0, 'acc': 0.75044, 'auc': np.float64(0.8333018787578815), 'dataset': 'HIGGS'}
Training RandomForest on HIGGS ...
{'model': 'RandomForest', 'fit_s': 797.721280316, 'pred_s': 10.282766372998594, 'proba_s': 10